In [1]:
%matplotlib widget
import isx 
import os 
import numpy as np
from scipy.fft import fft, ifft 
from scipy.ndimage import gaussian_filter 
import matplotlib.pyplot as plt
import tifffile
from tqdm import tqdm
import time
# libraries needed for the segmentation Freehand
from mpl_interactions import image_segmenter
import urllib
import PIL
from IPython.display import Image
import pandas as pd

In [2]:
# VERIFY TH EPATH OF YOUR FILES

notebook_path = os.getcwd()
print(notebook_path)



\\ocean.igf.internal\invivo\INSCOPIX\nVue\projet EMpit\NG2-Col1_F25\F25_LID_bis\LIDbis_1week


In [3]:
# Buils the loop to pre-read the files
movie_files = []

for file in os.listdir(notebook_path):
    if file.endswith(".isxd") or file.endswith(".raw"):
        full_path = os.path.join(notebook_path, file)
        movie_files.append(full_path)

# Sort alphabetically
movie_files = sorted(movie_files)

print(f"Found {len(movie_files)} movies")



Found 146 movies


In [4]:

# Read andExtract first frames 

first_frames = []

for file_path in tqdm(movie_files):

    try:
        movie = isx.Movie.read(file_path)
        
        # Extract first frame
        frame = movie.get_frame_data(0)
        
        first_frames.append(frame)

    except Exception as e:
        print(f"Error reading {file_path}: {e}")

# Convert to numpy array
first_frames = np.array(first_frames)

print("Final array shape:", first_frames.shape) # Verify the shape == first_frames.shape → (N_movies, height, width)

100%|████████████████████████████████████████████████████████████████████████████████| 146/146 [00:36<00:00,  4.00it/s]

Final array shape: (146, 800, 896)


In [5]:
pip install ImageIO ### we would need ImageIo library

  Using cached pillow-11.3.0-cp39-cp39-win_amd64.whl.metadata (9.2 kB)
Using cached pillow-11.3.0-cp39-cp39-win_amd64.whl (7.0 MB)

  Attempting uninstall: pillow

    Found existing installation: Pillow 8.0.1

    Uninstalling Pillow-8.0.1:

      Successfully uninstalled Pillow-8.0.1

   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   ---------------------------------------- 0/2 [pillow]
   -------------------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
isx 1.8.0 requires pillow==8.0.1, but you have pillow 11.3.0 which is incompatible.


In [8]:
pip install opencv-python

   ---------------------------------------- 0.0/40.2 MB ? eta -:--:--
   - -------------------------------------- 1.3/40.2 MB 6.7 MB/s eta 0:00:06
   ---- ----------------------------------- 4.5/40.2 MB 11.2 MB/s eta 0:00:04
   ------- -------------------------------- 7.1/40.2 MB 11.5 MB/s eta 0:00:03
   --------- ------------------------------ 9.7/40.2 MB 11.6 MB/s eta 0:00:03
   ----------- ---------------------------- 12.1/40.2 MB 11.6 MB/s eta 0:00:03
   -------------- ------------------------- 14.7/40.2 MB 11.7 MB/s eta 0:00:03
   ----------------- ---------------------- 17.3/40.2 MB 11.7 MB/s eta 0:00:02
   ------------------- -------------------- 19.9/40.2 MB 11.7 MB/s eta 0:00:02
   ---------------------- ----------------- 22.5/40.2 MB 11.8 MB/s eta 0:00:02
   ------------------------ --------------- 24.9/40.2 MB 11.8 MB/s eta 0:00:02
   --------------------------- ------------ 27.5/40.2 MB 11.7 MB/s eta 0:00:02
   ----------------------------- ---------- 30.1/40.2 MB 11.7 MB/s

In [9]:
import cv2

In [10]:
# Make your movie 

output_path = "first_frames_movie.avi" # choose the name 

processed_frames = []

for idx, frame in enumerate(first_frames):

    # Normalize to 8-bit
    frame_norm = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX)
    frame_uint8 = frame_norm.astype(np.uint8)

    # Convert to BGR
    frame_bgr = cv2.cvtColor(frame_uint8, cv2.COLOR_GRAY2BGR)

    # Add number in upper-right
    text = f"{idx+1}"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    thickness = 2

    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    text_x = frame_bgr.shape[1] - text_size[0] - 10
    text_y = 30

    cv2.putText(frame_bgr, text, (text_x, text_y),
                font, font_scale, (255,255,255),
                thickness, cv2.LINE_AA)

    processed_frames.append(frame_bgr)

# Get size
height, width, _ = processed_frames[0].shape

# Create video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 1  # change if needed

video = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

for frame in processed_frames:
    video.write(frame)

video.release()

print(f"AVI saved as {output_path}")

AVI saved as first_frames_movie.avi
